In [ ]:
import re, zlib, statistics, sys
from pathlib import Path
import math

for PDF in Path('../attachment').glob('*.pdf'):
    print('===', PDF.name)
    data = PDF.read_bytes()

    # 1) 抓取并解压（或保留）所有内容流
    streams = list(re.finditer(rb'stream\r?\n(.*?)\r?\nendstream', data, re.S))
    objs = []
    for m in streams:
        s = m.group(1)
        try:
            objs.append(zlib.decompress(s))
        except Exception:
            objs.append(s)

    # ---------------- Flag 1：折线里的 ASCII ----------------

    for s1 in objs:

        txt1 = s1.decode('latin1', 'ignore')

        # 提取「m ... l ... S」的最长一段（就是折线）
        blocks = list(re.finditer(r'([0-9.]+) ([0-9.]+) m\s*((?:[0-9.]+ [0-9.]+ l\s*)+)S', txt1))
        for blk in blocks:
            # 解析出 32 个点
            m = re.match(r'([0-9.]+) ([0-9.]+) m', blk.group(0))
            pts = [(float(m.group(1)), float(m.group(2)))]
            pts += [(float(x), float(y)) for x, y in re.findall(r'([0-9.]+) ([0-9.]+) l', blk.group(0))]
            ys = [y for _, y in pts]

            # y 与 ASCII 是线性关系：用前两点假设开头是 "fl"（CTF 常见 "flag{"），快速标定
            a = (ys[1] - ys[0]) / (math.log(108) - math.log(102))    # 每 1 个 ASCII 的 y 增量
            if a<1e-10:
                #print('!!bad', a)
                continue
            b = ys[0] - a * math.log(102)                  # 截距
            flag1 = ''.join(chr(round(math.exp((y - b) / a))) for y in ys)
            if flag1.startswith('flag{'):
                print(flag1)

    for s2 in objs:
        txt2 = s2.decode('latin1', 'ignore')
        moves = [(float(x), float(y)) for x, y in
                re.findall(r'1 0 0 1 ([\-\d.]+) ([\-\d.]+) cm /M\d+ Do', txt2)]

        if not moves:
            continue

        # 自动估计网格步长
        sx = max(abs(dx) for dx, dy in moves if abs(dx) > 1e-6) / 3
        sy = max(abs(dy) for dx, dy in moves if abs(dy) > 1e-6) / 3

        # 累积平移 -> 整数格坐标
        x = y = 0
        pos = []
        for i, (dx, dy) in enumerate(moves):
            if i:
                x += round(dx / sx)
                y += round(dy / sy)
            pos.append((x, y))

        # 平移到 [0..3]×[0..3]
        minx = min(px for px, py in pos); miny = min(py for px, py in pos)
        adj = [(px - minx, py - miny) for px, py in pos]

        # (x, y) -> nibble：value = x + 4*y
        hexvals = [x + 4 * y for x, y in adj]
        hexstr = ''.join('0123456789abcdef'[v] for v in hexvals)
        flag2 = bytes.fromhex(hexstr).decode()
        print(flag2)